# Advanced Neural Net Regression Example 
This example will show you how to use one of the provided data sets and ```amorf.NeuralNetRegression.NeuralNetRegressor``` to perform a multi-output Regression using a custom PyTorch model. 


## Load Test and Training Data

In [25]:
from amorf.datasets import RiverFlow1 
from sklearn.model_selection import train_test_split

X, y = RiverFlow1().get_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

## Normalizing Data

In [27]:
from sklearn.preprocessing import normalize 

X_train = normalize(X_train) 
X_test = normalize(X_test)

## Build Custom PyTorch Model 

If you feel you don't know enough about PyTorch moodels you can take a look at this https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py  

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, input_dim, output_dim, p_dropout_1=0.5, p_dropout_2=0.5):
        super().__init__()
        self.dropout_1 = p_dropout_1
        self.dropout_2 = p_dropout_2
       
        self.batchNorm1 = nn.BatchNorm1d(256)
        self.batchNorm2 = nn.BatchNorm1d(64)

        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64) 
        self.fc4 = nn.Linear(64,32) 
        self.fc5 = nn.Linear(32,output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.batchNorm1(out)
        out = F.relu(out)
        out = F.dropout(out, self.dropout_1)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        out = self.batchNorm2(out)
        out = F.relu(out) 
        out = F.dropout(out, self.dropout_2)
        out = self.fc4(out)
        out = F.relu(out) 
        out = self.fc5(out)

        return out 
    
    def convert_train_set_to_tensor(self, X_train, y_train, device):
        X_train_t = torch.from_numpy(X_train).to(device).float()
        y_train_t = torch.from_numpy(y_train).to(device).float()

        return X_train_t, y_train_t

    def convert_test_set_to_tensor(self, X_test, device):
        X_test_t = torch.from_numpy(X_test).to(device).float()
        return X_test_t

## Initialize and Train Estimator

In [11]:
import amorf.neuralNetRegression as neural 

input_dim = len(X_train[0])
output_dim = len(y_train[0]) 

model = Net(input_dim, output_dim, p_dropout_1=0.3, p_dropout_2=0.2)  
estimator = neural.NeuralNetRegressor(model= model, patience=None, training_limit=1000, batch_size=1000) 
estimator.fit(X_train, y_train)


Epoch: 0
Validation Error: 2.8269431591033936 
Train Error: 2.438387393951416
Epoch: 100
Validation Error: 0.445943146944046 
Train Error: 0.41736263036727905
Epoch: 200
Validation Error: 0.3839591145515442 
Train Error: 0.3855123221874237
Epoch: 300
Validation Error: 0.35046833753585815 
Train Error: 0.34234800934791565
Epoch: 400
Validation Error: 0.31788256764411926 
Train Error: 0.32208138704299927
Epoch: 500
Validation Error: 0.3051885962486267 
Train Error: 0.3064659833908081
Epoch: 600
Validation Error: 0.2929025888442993 
Train Error: 0.30002111196517944
Epoch: 700
Validation Error: 0.2793003022670746 
Train Error: 0.2870289087295532
Epoch: 800
Validation Error: 0.27429503202438354 
Train Error: 0.2892204225063324
Epoch: 900
Validation Error: 0.2604061961174011 
Train Error: 0.27726155519485474
Epoch: 1000
Validation Error: 0.2517169713973999 
Train Error: 0.26893192529678345
Epoch: 1100
Validation Error: 0.23829714953899384 
Train Error: 0.25703567266464233
Epoch: 1200
Validat

## Perform Prediction and Calculate Error 


In [13]:
from amorf.metrics import average_relative_root_mean_squared_error

result = estimator.predict(X_test) 
print(average_relative_root_mean_squared_error(result, y_test))

0.19865390937775373


## Save Model

In [22]:
path = "savedExampleModel"
estimator.save(path)

/home/davidhildner/anaconda3/envs/ppl/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/davidhildner/anaconda3/envs/ppl/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/davidhildner/anaconda3/envs/ppl/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


## Load Model and predict

In [24]:
newEstimator = neural.NeuralNetRegressor(patience=None, training_limit=10000, batch_size=1000) 
newEstimator.load("savedExampleModel.cpkt") 

new_result = newEstimator.predict(X_test) 
print(average_relative_root_mean_squared_error(result, y_test))

Traceback (most recent call last):
  File "/home/davidhildner/Dropbox/Masterarbeit/01_Research/Code/david_master_thesis/amorf/neuralNetRegression.py", line 176, in load
    model = torch.load(load_path).to(self.Device)
  File "/home/davidhildner/anaconda3/envs/ppl/lib/python3.7/site-packages/torch/serialization.py", line 419, in load
    f = open(f, 'rb')
FileNotFoundError: [Errno 2] No such file or directory: 'savedExampleModel.cpkt'



AttributeError: 'NoneType' object has no attribute 'convert_test_set_to_tensor'